In [1]:
import requests
import json
import re
import time
import pandas as pd
from bs4 import BeautifulSoup
from discord import Webhook, RequestsWebhookAdapter

In [2]:
base_url = "https://www.google.com/finance/quote/"
discord_webhook_url = "https://discord.com/api/webhooks/940522536557498368/un_14JP-n7T6haQzIhsDB9Dfj_XjWB_2wYFK6oD-opeC-ZfRhgH29_LSbWnNGnOzGi5B"

In [3]:
def fetch_data(stock):
    final_url = base_url + stock
    data = {}
    html_data = requests.get(final_url).text
    soup = BeautifulSoup(html_data, "html.parser")
    data["title"] = soup.title.string

    current_price = soup.find('div', class_='YMlKec fxKbKc').text
    current_price = re.findall('\d+\.\d+', current_price)
    data['current_price'] = float(current_price[0])

    sidebar_data = soup.find('div', 'eYanAe').text
    sidebar_data = re.findall('\d+\.\d+', sidebar_data)
    sidebar_data = [float(data) for data in sidebar_data]

    data['previous_close'] = sidebar_data[0]
    data['day_min'] = sidebar_data[1]
    data['day_max'] = sidebar_data[2]
    data['year_min'] = sidebar_data[3]
    data['year_max'] = sidebar_data[4]
    data['market_cap'] = sidebar_data[5]

    return data

In [4]:
def send_meeage_to_discord(message):
    webhook = Webhook.from_url(discord_webhook_url, adapter=RequestsWebhookAdapter())
    webhook.send(message)

In [7]:
def update_config(data):
    with open('config.json', 'w') as f:
        json.dump(data, f)

def read_config():
    with open('config.json', 'r') as f:
        configs = json.load(f)
    
    return configs

In [9]:
while True:
    time.sleep(300)
    configs = read_config()
    
    for config in configs:
        stock_data = fetch_data(config['stock'])
        difference = config['min_price']- stock_data['current_price']
        if stock_data['current_price'] < config['min_price']:
            message = '{} is currently down at: {}, min threshold: {}, difference: {}'.format(
                                config['stock_name'], 
                                stock_data['current_price'],
                                config['min_price'], 
                                difference)
            config['min_price'] = stock_data['current_price']
            send_meeage_to_discord(message)
            update_config(configs)
            print(message)
        if stock_data['current_price'] > config['max_price']:
            difference = stock_data['current_price']- config['max_price']
            message = '{} is currently Up at: {}, min threshold: {}, difference: {}'.format(
                                config['stock_name'], 
                                stock_data['current_price'],
                                config['max_price'], 
                                difference)
            
            config['max_price'] = stock_data['current_price']
            send_meeage_to_discord(message)
            update_config(configs)
            print(message)

TRIDENT is currently down at: 60.15, min threshold: 61, difference: 0.8500000000000014


KeyboardInterrupt: 